In [1]:
import pandas as pd
from datetime import datetime, timedelta
import calendar
import datetime
from pymongo import MongoClient
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
import numpy
import json
from bson.json_util import dumps

#############################################################
#This is a simple dummy program created to learn more about python based iteration and handling
#of data. Does not have any PII business data or business logic copied.
#############################################################
#Establisth MongoDB connection
client = MongoClient('mongodb://localhost:27017/')
db = client.Forecast

forecasts = db.salesforecasts

#Delete all existing Data in Forecast Collection
x = forecasts.delete_many({})
print(x.deleted_count, " documents deleted.")

#############################################################
#This extracts the values of ARIMA order p,d,q from the DB 
modelparams = db.modelparams

modelparamsdocs = modelparams.find({"model": "ARIMA"}, {"values": 1, "_id": False})

for cursor in modelparamsdocs:
  print(cursor.get("values"))
  for val in cursor.get("values"):
     if val.get("key") == 'p':
       p = val.get("value")
     if val.get("key") == 'd':
       d = val.get("value")
     if val.get("key") == 'q':
       q = val.get("value")
            
#############################################################
#This function calculates all the week start dates of an year
#futureYearWeekStarts = ['06/Jan/2020', '13/Jan/2020', '20/Jan/2020', '27/Jan/2020', '03/Feb/2020', '10/Feb/2020', '17/Feb/2020', '24/Feb/2020', '02/Mar/2020', '09/Mar/2020', '16/Mar/2020', '23/Mar/2020', '30/Mar/2020', '06/Apr/2020', '13/Apr/2020', '20/Apr/2020', '27/Apr/2020', '04/May/2020', '11/May/2020', '18/May/2020', '25/May/2020', '01/Jun/2020', '08/Jun/2020', '15/Jun/2020', '22/Jun/2020', '29/Jun/2020', '06/Jul/2020', '13/Jul/2020', '20/Jul/2020', '27/Jul/2020', '03/Aug/2020', '10/Aug/2020', '17/Aug/2020', '24/Aug/2020', '31/Aug/2020', '07/Sep/2020', '14/Sep/2020', '21/Sep/2020', '28/Sep/2020', '05/Oct/2020', '12/Oct/2020', '19/Oct/2020', '26/Oct/2020', '02/Nov/2020', '09/Nov/2020', '16/Nov/2020', '23/Nov/2020', '30/Nov/2020', '07/Dec/2020', '14/Dec/2020', '21/Dec/2020', '28/Dec/2020']
# This function calculates all the week start dates of an year
futureYearWeekStarts = []
def get_start_and_end_date_from_calendar_week(year, calendar_week):       
    monday = datetime.datetime.strptime(f'{year}-{calendar_week}-1', "%Y-%W-%w").date()
    return monday.strftime('%d/%b/%Y')

for x in range(1, 53):
   futureYearWeekStarts.append(get_start_and_end_date_from_calendar_week(2020, x))
#############################################################
#This function creates individual docs to be inserted in the collection
def merge(list1, list2, prod, chn, attr): 
    merged_list = []  
    for i in range(0, len(list1)):
        doc_obj = {"channel": chn,
                 "saleDate": list1[i],
                 "producct": prod,
                 "attribute": attr,
                 "quantity": list2[i]}
        merged_list.append(doc_obj)  
    return merged_list 

#############################################################

# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        #print(dataset[i])
        diff.append(value)
    return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

series = read_csv('/Users/parnabbasak/Documents/workspace/Group3_Hackathon/Node_Services/uploads/sales_history.csv')

df = pd.DataFrame(series)

#Find unique channels
uniquechannels = df['channel'].unique()

#Iterating over unique channels
for i in uniquechannels: 
    
    #Get data for every unique channels 
    on_channel = df[df['channel'] == i]
    
    #Find unique products for each channel
    uniqueproducts = on_channel.producct.unique()
    
    #Iterating over every unique product for a specific channel
    for j in uniqueproducts:
        
        #Get data for every unique product for a specific channel
        on_channel_and_product = on_channel[on_channel['producct'] == j]
        
        #Only select attribute = NET since that would be used for ARIMA calculation
        on_channel_and_product_attribute = on_channel_and_product[on_channel_and_product['attribute'] == 'Net Unit']
        #print(on_channel_and_product_attribute)
        
        X = on_channel_and_product_attribute["quantity"].tolist()
        weeks_in_year = 52
        differenced = difference(X, weeks_in_year)
        
        # fit model
        model = ARIMA(differenced, order=(1,1,1))
        model_fit = model.fit(disp=0)
        # multi-step out-of-sample forecast
        forecast = model_fit.forecast(steps=52)[0]
        
        # invert the differenced forecast to something usable
        history = [x for x in X]
        #history
        week = 1
        predictions = []
        for yhat in forecast:
            inverted = inverse_difference(history, yhat, weeks_in_year)
            #print('Week %d: %f' % (week, inverted))
            history.append(inverted)
            predictions.append(inverted)
            week += 1
        merged_forecast = merge(futureYearWeekStarts, predictions, j, i, 'ARIMA')
        
        x = forecasts.insert_many(merged_forecast)
        print(merged_forecast)
        print('----Inserted x.inserted_ids elements in the  collection------------------------------------------------------')
        

52  documents deleted.
[{'channel': 'DSO', 'saleDate': '06/Jan/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 109.0, '_id': ObjectId('5dc3f5537cff32b656bd0688')}, {'channel': 'DSO', 'saleDate': '13/Jan/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 117.0, '_id': ObjectId('5dc3f5537cff32b656bd0689')}, {'channel': 'DSO', 'saleDate': '20/Jan/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 111.0, '_id': ObjectId('5dc3f5537cff32b656bd068a')}, {'channel': 'DSO', 'saleDate': '27/Jan/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 112.0, '_id': ObjectId('5dc3f5537cff32b656bd068b')}, {'channel': 'DSO', 'saleDate': '03/Feb/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 113.0, '_id': ObjectId('5dc3f5537cff32b656bd068c')}, {'channel': 'DSO', 'saleDate': '10/Feb/2020', 'producct': 'Brand A1-Phone', 'attribute': 'ARIMA', 'quantity': 114.0, '_id': ObjectId('5dc3f5537cff32b656bd068d')}

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/tsa/ar_model.py:763: RuntimeWarning: divide by zero encountered in log
  return np.log(self.sigma2) + (1 + self.df_model) * np.log(nobs)/nobs
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [60]:
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
import numpy

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# load dataset
series = read_csv('../uploads/saleshistory.csv', header=None)
# seasonal difference
X = series.values
days_in_year = 365
differenced = difference(X, days_in_year)
# fit model
model = ARIMA(differenced, order=(7,0,1))
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
forecast = model_fit.forecast(steps=100)[0]
# invert the differenced forecast to something usable
history = [x for x in X]
day = 1
for yhat in forecast:
	inverted = inverse_difference(history, yhat, days_in_year)
	print('Day %d: %f' % (day, inverted))
	history.append(inverted)
	day += 1

Day 1: 14.861671
Day 2: 15.628707
Day 3: 13.331307
Day 4: 11.722377
Day 5: 10.421468
Day 6: 14.415496
Day 7: 12.674673
Day 8: 14.750578
Day 9: 13.255427
Day 10: 15.570908
Day 11: 14.483631
Day 12: 14.291933
Day 13: 15.297140
Day 14: 16.400200
Day 15: 14.802079
Day 16: 17.403600
Day 17: 18.805085
Day 18: 22.106500
Day 19: 19.007745
Day 20: 15.508778
Day 21: 15.809606
Day 22: 14.710260
Day 23: 10.710779
Day 24: 11.511198
Day 25: 15.011542
Day 26: 14.511825
Day 27: 14.512059
Day 28: 13.312250
Day 29: 14.312408
Day 30: 14.312536
Day 31: 20.512640
Day 32: 15.012725
Day 33: 17.112795
Day 34: 16.912852
Day 35: 16.912899
Day 36: 13.612937
Day 37: 16.412968
Day 38: 16.112993
Day 39: 12.013014
Day 40: 12.213031
Day 41: 14.813045
Day 42: 14.813056
Day 43: 14.413066
Day 44: 12.913073
Day 45: 13.413079
Day 46: 15.913084
Day 47: 16.113089
Day 48: 17.613092
Day 49: 15.613095
Day 50: 15.013097
Day 51: 13.013099
Day 52: 14.113100
Day 53: 17.313102
Day 54: 15.713103
Day 55: 18.613103
Day 56: 12.713104
D

In [70]:
from datetime import datetime, timedelta
 
date_str = '2020-01-01'
date_obj = datetime.strptime(date_str, '%Y-%m-%d')
 
start_of_week = date_obj - timedelta(days=date_obj.weekday())  # Monday
end_of_week = start_of_week + timedelta(days=6)  # Sunday
print(start_of_week)
print(end_of_week)

2019-12-30 00:00:00
2020-01-05 00:00:00
